In [1]:
pip install langchain_community langchain_openai faiss-cpu pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:

#### chatGPT RAG 활용
import os
import openai
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain, LLMChain, StuffDocumentsChain
# from langchain.retrievers import EmbeddingRetriever
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader, TextLoader
import numpy as np
####
from dotenv import load_dotenv
import json
import faiss
from openai import OpenAI


#### api 키 설정
api_key = "api_key"
os.environ['OPENAI_API_KEY'] = api_key
####




client = OpenAI(
    api_key=api_key,
)

def generate_text(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"system", "content":""}
            {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


In [3]:
# 1. 디렉토리 내의 모든 .txt 파일을 불러오기
loader = DirectoryLoader("./지문 관련", glob="*.txt", loader_cls=TextLoader)
documents = loader.load()

In [4]:
documents

[Document(metadata={'source': '지문 관련\\26수특_과학분야의 글이란.txt'}, page_content='- 자연 과학적 시각으로 물질계와 생태계, 우주를 탐구하는 인간의 정신 활동을 담고 있는 글이다.\n- 수에 관하여 연구하는 수학, 물질의 물리적 성질과 운동 형태 등을 연구하는 물리학, 물질의 조성과 구조 · 성질 등을 연구하는 화학, 생물의 구조와 기능을 과학적으로 연구하는 생명 과학, 지구 및 천체를 연구하는 지구 과학과 관련된 글 등을 포함한다.'),
 Document(metadata={'source': '지문 관련\\26수특_과학분야의 글읽기방법.txt'}, page_content='- 과학 분야의 글은 낯선 용어와 원리, 방법이 제시되어 있는 경우가 많기 때문에 용어의 개념과 원리, 방법에 관한 정보를 이해하는 것을 중시해야 한다.\n- 과학 이론을 설명하는 글의 경우, 과학자가 제시한 개념과 원리, 방법, 견해, 주장에 관한 정보에 특히 주목해야 하며, 과학 현상을 설명하는 경우, 과정, 개념, 원리, 방법에 관한 정보에 특히 주목해야 한다.\n- 특정 과학 분야를 통시적 관점에서 다루는 글을 읽을 때에는 과학자들이 제시한 이론이 시간의 흐름에 따라 어떻게 달라졌는지를 파악하여야 한다.'),
 Document(metadata={'source': '지문 관련\\26수특_과학분야의 출제경향_지문.txt'}, page_content='- 물리학, 화학, 생명 과학, 지구 과학 분야의 과학적 원리와 탐구 과정, 실험 과정 등을 다룬 지문이 출제되고 있다.\n- 화제에 대한 개념을 설명하고 그에 대한 이해를 바탕으로 원리, 방법을 이해해야 하는 지문이 자주 출제되고 있다.\n- 최근 자주 출제되고 있는 생명 과학 제재의 경우, 구성 요소의 기능과 특징을 개념, 원리, 방법 등에 관한 정보와 함께 중시해야 하는 지문이 출제되고 있다.'),
 Document(metadata={'source': '지문 관련\\26수특_기술분야의 글

In [5]:
# 2. OpenAI 임베딩 모델 초기화
embeddings_model = OpenAIEmbeddings()

In [12]:
# 3. 문서 내용을 벡터로 변환
texts = [doc.page_content for doc in documents]
metadata = [doc.metadata for doc in documents]  # 파일명, 경로 등 메타데이터 저장
embeddings = embeddings_model.embed_documents(texts)

# 4. FAISS 인덱스 생성
vector_db = FAISS.from_texts(texts, embeddings_model, metadatas=metadata)

# 5. FAISS 인덱스를 파일로 저장
faiss_index_file_path = "./faiss_index_txt_files"
vector_db.save_local(faiss_index_file_path)

print(f"FAISS 벡터 DB 저장 완료: {faiss_index_file_path}")

FAISS 벡터 DB 저장 완료: ./faiss_index_txt_files


In [16]:
# 저장된 FAISS 인덱스 로드
vector_db = FAISS.load_local("faiss_index_txt_files", embeddings_model, allow_dangerous_deserialization=True)

# 검색할 쿼리
query = "과학"

# 유사한 문서 3개 검색
results = vector_db.similarity_search(query, k=3)

# 필수 문서 2개를 검색 결과에 포함시키기 위해 직접 검색
required_files = ["./지문 관련/자료작성원리.txt", "./지문 관련/평가문항예제.txt"]

# FAISS의 내부 docstore에서 필수 문서 가져오기
required_docs = []
for doc in vector_db.docstore._dict.values():  # FAISS 내부 저장소 순회
    if "source" in doc.metadata and doc.metadata["source"] in required_files:
        required_docs.append(doc)

# 중복 제거 (필수 문서가 이미 검색된 문서에 포함되지 않도록 처리)
unique_results = {res.page_content: res for res in results}  # 기존 검색된 문서 저장
for doc in required_docs:
    if doc.page_content not in unique_results:
        unique_results[doc.page_content] = doc  # 필수 문서 추가

# 최종 결과 리스트 (최대 5개)
final_results = list(unique_results.values())[:5]

# 결과 출력
for i, res in enumerate(final_results):
    print(f"\n[{i+1}] 검색된 문서:")
    print(res.page_content)
    print("Metadata:", res.metadata)



🔴 필수 문서가 FAISS에서 누락되었습니다. 다시 추가하세요: {'./지문 관련/자료작성원리.txt', './지문 관련/평가문항예제.txt'}

[1] 검색된 문서:
- 자연 과학적 시각으로 물질계와 생태계, 우주를 탐구하는 인간의 정신 활동을 담고 있는 글이다.
- 수에 관하여 연구하는 수학, 물질의 물리적 성질과 운동 형태 등을 연구하는 물리학, 물질의 조성과 구조 · 성질 등을 연구하는 화학, 생물의 구조와 기능을 과학적으로 연구하는 생명 과학, 지구 및 천체를 연구하는 지구 과학과 관련된 글 등을 포함한다.
Metadata: {'source': '지문 관련\\26수특_과학분야의 글이란.txt'}

[2] 검색된 문서:
- 물리학, 화학, 생명 과학, 지구 과학 분야의 과학적 원리와 탐구 과정, 실험 과정 등을 다룬 지문이 출제되고 있다.
- 화제에 대한 개념을 설명하고 그에 대한 이해를 바탕으로 원리, 방법을 이해해야 하는 지문이 자주 출제되고 있다.
- 최근 자주 출제되고 있는 생명 과학 제재의 경우, 구성 요소의 기능과 특징을 개념, 원리, 방법 등에 관한 정보와 함께 중시해야 하는 지문이 출제되고 있다.
Metadata: {'source': '지문 관련\\26수특_과학분야의 출제경향_지문.txt'}

[3] 검색된 문서:
- 인간의 삶을 편리하게 하는 산업 기술, 생활 기술 등 다양한 분야의 기술을 설명하는 글이다.
- 특정 과학 이론을 바탕으로 장치나 시스템에 적용되는 원리와 작동 과정, 한계 등을 구체적으로 서술한 글이다.
- 전기와 전자의 원리를 이용한 공학 기술, 컴퓨터를 이용한 공학 기술, 화학이나 생명 과학과 결합된 공학 기술, 토목이나 건축에 활용되는 토목건축 공학 기술과 관련된 글 둥을 포함한다.
Metadata: {'source': '지문 관련\\26수특_기술분야의 글이란.txt'}


In [11]:
# 저장된 FAISS 인덱스 로드
vector_db = FAISS.load_local("faiss_index_txt_files", embeddings_model, allow_dangerous_deserialization=True)

# 검색할 쿼리
query = "과학"

# 유사한 문서 3개 검색
results = vector_db.similarity_search(query, k=3)

# 필수적으로 포함할 문서 2개 (예시: 특정 문서 ID를 통해 가져오기)
required_docs = [
    vector_db.docstore.search("./지문 관련/자료작성원리.txt"),
    vector_db.docstore.search("./지문 관련/평가문항예제.txt")
]

# 중복 제거 (필수 문서가 이미 검색 결과에 포함되었는지 확인)
unique_results = {res.page_content: res for res in results}  # 검색된 문서 집합

for doc in required_docs:
    if doc and doc.page_content not in unique_results:
        unique_results[doc.page_content] = doc  # 필수 문서 추가

# 최종 결과 리스트 (최대 5개)
final_results = list(unique_results.values())[:5]

# 결과 출력
for i, res in enumerate(final_results):
    print(f"\n[{i+1}] 검색된 문서:")
    print(res.page_content)
    print("Metadata:", res.metadata)



AttributeError: 'str' object has no attribute 'page_content'

In [17]:
results

[Document(id='e9579243-d28c-4653-a939-20b38df164d3', metadata={'source': '지문 관련\\26수특_과학분야의 글이란.txt'}, page_content='- 자연 과학적 시각으로 물질계와 생태계, 우주를 탐구하는 인간의 정신 활동을 담고 있는 글이다.\n- 수에 관하여 연구하는 수학, 물질의 물리적 성질과 운동 형태 등을 연구하는 물리학, 물질의 조성과 구조 · 성질 등을 연구하는 화학, 생물의 구조와 기능을 과학적으로 연구하는 생명 과학, 지구 및 천체를 연구하는 지구 과학과 관련된 글 등을 포함한다.'),
 Document(id='f11f2876-d96f-4f05-b9db-82e9955d08e5', metadata={'source': '지문 관련\\26수특_과학분야의 출제경향_지문.txt'}, page_content='- 물리학, 화학, 생명 과학, 지구 과학 분야의 과학적 원리와 탐구 과정, 실험 과정 등을 다룬 지문이 출제되고 있다.\n- 화제에 대한 개념을 설명하고 그에 대한 이해를 바탕으로 원리, 방법을 이해해야 하는 지문이 자주 출제되고 있다.\n- 최근 자주 출제되고 있는 생명 과학 제재의 경우, 구성 요소의 기능과 특징을 개념, 원리, 방법 등에 관한 정보와 함께 중시해야 하는 지문이 출제되고 있다.'),
 Document(id='0a3fa8c5-9ecc-4380-90ca-d33fecb49385', metadata={'source': '지문 관련\\26수특_기술분야의 글이란.txt'}, page_content='- 인간의 삶을 편리하게 하는 산업 기술, 생활 기술 등 다양한 분야의 기술을 설명하는 글이다.\n- 특정 과학 이론을 바탕으로 장치나 시스템에 적용되는 원리와 작동 과정, 한계 등을 구체적으로 서술한 글이다.\n- 전기와 전자의 원리를 이용한 공학 기술, 컴퓨터를 이용한 공학 기술, 화학이나 생명 과학과 결합된 공학 기술, 토목이나 건축에 활용되는 토목건축 공학 기술과 관련된 글 

In [18]:
for i, res in enumerate(results):
    print(f"\n[{i+1}] 검색된 문서:")
    print("내용:", res.page_content)
    print("메타데이터:", res.metadata)  # 메타데이터 조회



[1] 검색된 문서:
내용: - 자연 과학적 시각으로 물질계와 생태계, 우주를 탐구하는 인간의 정신 활동을 담고 있는 글이다.
- 수에 관하여 연구하는 수학, 물질의 물리적 성질과 운동 형태 등을 연구하는 물리학, 물질의 조성과 구조 · 성질 등을 연구하는 화학, 생물의 구조와 기능을 과학적으로 연구하는 생명 과학, 지구 및 천체를 연구하는 지구 과학과 관련된 글 등을 포함한다.
메타데이터: {'source': '지문 관련\\26수특_과학분야의 글이란.txt'}

[2] 검색된 문서:
내용: - 물리학, 화학, 생명 과학, 지구 과학 분야의 과학적 원리와 탐구 과정, 실험 과정 등을 다룬 지문이 출제되고 있다.
- 화제에 대한 개념을 설명하고 그에 대한 이해를 바탕으로 원리, 방법을 이해해야 하는 지문이 자주 출제되고 있다.
- 최근 자주 출제되고 있는 생명 과학 제재의 경우, 구성 요소의 기능과 특징을 개념, 원리, 방법 등에 관한 정보와 함께 중시해야 하는 지문이 출제되고 있다.
메타데이터: {'source': '지문 관련\\26수특_과학분야의 출제경향_지문.txt'}

[3] 검색된 문서:
내용: - 인간의 삶을 편리하게 하는 산업 기술, 생활 기술 등 다양한 분야의 기술을 설명하는 글이다.
- 특정 과학 이론을 바탕으로 장치나 시스템에 적용되는 원리와 작동 과정, 한계 등을 구체적으로 서술한 글이다.
- 전기와 전자의 원리를 이용한 공학 기술, 컴퓨터를 이용한 공학 기술, 화학이나 생명 과학과 결합된 공학 기술, 토목이나 건축에 활용되는 토목건축 공학 기술과 관련된 글 둥을 포함한다.
메타데이터: {'source': '지문 관련\\26수특_기술분야의 글이란.txt'}


In [20]:
topic = "인공지능과 기계학습"
received_text = f'''
"{topic}" 주제를 바탕으로, 한국교육과정평가원 스타일의 문제 풀이를 위한 독서 지문을 작성하세요.

[분야]
인문, 예술, 사회문화, 기술, 과학, 주제통합

[출제 기준 참고]
다음은 "{topic}" 관련 수학능력시험 국어영역 독서과목 출제 경향, 자료 작성 원리, "{topic}"에 해당하는 분야별 글에 대한 설명, 지문 작성 가이드라인입니다.
이 기준을 충실히 반영하여 지문을 작성하십시오.

{results}

[작성 조건]
- 글자 수: 1200자 내외 (±100자)
- 문체: 문어체, 논리적, 객관적 서술
- 전개 방식: 서론 → 본론 → 결론의 구조

다음은 독서 지문의 예시입니다.
[지문 예시]
문장이나 영상, 음성을 만들어 내는 인공 지능 생성 모델 중확산 모델은 영상의 복원, 생성 및 변환에 뛰어난 성능을 보인다.
확산 모델의 기본 발상은, 원본 이미지에 노이즈를 점진적으로 추가하였다가 그 노이즈를 다시 제거해 나가면 원본 이미지를 복원할 수 있다는 것이다. 노이즈는 불필요하거나 원하지 않는 값을 의미한다. 원하는 값만 들어 있는 원본 이미지에 노이즈를 단계별로 더하면 노이즈가 포함된 확산 이미지가 되고, 여러 단계를 거치면 결국 원본 이미지가 어떤 이미지였는지 전혀 알아볼 수 없는 노이즈 이미지가 된다. 역으로, 단계별로 더해진 노이즈를 알 수 있다면 노이즈 이미지에서 원본 이미지를 복원할수 있다. 확산 모델은 노이즈 생성기, 이미지 연산기, 노이즈 예측기로 구성되며, 순확산 과정과 역확산 과정 순으로 작동한다.
순확산 과정은 이미지에 노이즈를 추가하면서 노이즈 예측기를 학습시키는 과정이다. 첫 단계에서는, 노이즈 생성기에서 노이즈를 만든 후 이미지 연산기가 이 노이즈를 원본 이미지에 더해서 노이즈가 포함된 확산 이미지를 출력한다. 다음 단계부터는 노이즈 생성기에서 만든 노이즈를 이전 단계에서 출력된 확산 이미지에 더한다. 이러한 단계를 충분히 반복하면 최종적으로 노이즈 이미지가 출력된다. 이때 더해지는 노이즈는 크기나 분포 양상 등 그 특성이 단계별로 다르다. 따라서 노이즈 예측기는 단계별로 확산 이미지를 입력받아 이미지에 포함된 노이즈의 특성을 추출하여 수치들로 표현하고, 이 수치들을 바탕으로 노이즈를 예측한다. 노이즈 예측기 내부의 이러한 수치들을
잠재 표현 이라고 한다. 노이즈 예측기는 잠재 표현을 구하고 노이즈를 예측하는 방식을 학습한다.
노이즈 예측기의 학습 방법은 기계 학습 중에서 지도 학습에 해당한다. 지도 학습은 학습 데이터에 정답이 주어져 출력과 정답의 차이가 작아지도록 모델을 학습시키는 방법이다. 노이즈 예측기를 학습시킬 때는 노이즈 생성기에서 만들어 넣어 준노이즈가 정답에 해당하며 이 노이즈와 예측된 노이즈 사이의 차이가 작아지도록 학습시킨다.
역확산 과정은 노이즈 이미지에서 노이즈를 제거하여 원본 이미지를 복원하는 과정이다. 노이즈를 제거하려면 이미지에 단계별로 어떤 특성의 노이즈가 더해졌는지 알아야 하는데 노이즈 예측기가 이 역할을 한다. 노이즈 이미지 또는 중간 단계에서의 확산 이미지를 노이즈 예측기에 입력하면 이미지에 포함된 노이즈의 특성을 추출하여 잠재 표현을 구하고 이를 바탕으로 노이즈를 예측한다. 이미지 연산기는 입력된 확산 이미지로부터 이 노이즈를 빼서 현 단계의 노이즈를 제거한 확산 이미지를 출력한다. 확산 이미지에 이런 단계를 반복하면 결국 노이즈가 대부분 제거되어 원본 이미지에 가까운 이미지만 남게 된다.
한편, 많은 종류의 이미지를 학습시킨 후 학습된 이미지의 잠재 표현에 고유 번호를 붙이면 역확산 과정에서 이미지를 선택하여 생성할 수 있다. 또한 잠재 표현의 수치들을 조정하면 다른 특성의 노이즈가 생성되어 여러 이미지를 혼합하거나 실재하지 않는 이미지를 만들어 낼 수도 있다.

[핵심 논점(출제 의도)]
생성된 지문에서 평가원 출제위원의 시각에서 학생이 반드시 이해해야 할 핵심 논점 2~3개를 요약해 제시한 것을 기억해 두세요. (각 1~2문장씩)
논점은 반드시 지문 내 핵심 문장이나 개념, 논리 전개에 근거해 작성할 것.

[금지 사항]
- 문학 작품 생성 금지.
- 허구적 사건이나 사람 이름 쓰지 말고, 객관적 사실 중심으로.
- 문장은 반드시 완결성을 갖출 것. 어색한 비문 생성 금지.


'''

In [21]:
llm_result = generate_text(received_text)

In [22]:
llm_result

"[지문 작성]\n\n인공지능(AI)과 기계학습은 현대 사회에서 다양한 분야에 변화를 가져오고 있다. 이러한 기술의 발전은 많은 분야에서 인간의 삶을 편리하게 하는 데 크게 기여하고 있다. 인공지능은 사람의 지능적 행동을 모방하고, 미리 설정된 규칙 없이도 데이터를 기반으로 학습한다는 점에서 독특한 특성을 지닌다. 이와 함께 기계학습은 대량의 데이터를 분석하고 이를 기반으로 학습 가능한 모델을 구축하여 활용하는 데 중점을 둔다. 이러한 원리를 깊이 이해하는 것은 더욱 발전된 응용을 위해 필수적이다.\n\nAI와 기계학습은 다양한 학문적 영역에서 샘솟는 혁신적 연구의 토대를 제공한다. 기계학습의 기본 개념은 '훈련'과 '예측'의 과정으로 설명할 수 있다. 훈련 과정에서는 주어진 데이터 세트를 활용하여 모델을 구축한다. 이때, 데이터는 입력 값과 정답 값으로 구성되며, 모델은 입력 값을 결과로 변환하는 함수로서 학습한다. 특별히, 지도 학습(supervised learning)과 비지도 학습(unsupervised learning)이라는 두 가지 주요 학습 방식이 있다. 지도 학습에서는 정답을 포함한 데이터 세트가 필요하며, 모델은 이를 통해 정확성을 향상시킨다. 반면, 비지도 학습은 정답 없는 데이터 세트로부터 패턴을 인식하며, 데이터를 군집화하거나 분류할 수 있는 잠재 구조를 찾는다.\n\nAI와 기계학습의 발전은 다양한 분야에서 사람들에게 실질적인 혜택을 제공하고 있다. 예를 들어, 의료 분야에서는 질병 진단과 치료의 정확성을 높이고, 금융 분야에서는 정확한 시장 예측과 사기 탐지를 가능하게 한다. 또한, AI 기반의 자연어 처리 기술은 사람과 컴퓨터 간의 상호 작용을 보다 원활하게 하여, 사용자가 필요로 하는 정보를 효율적으로 제공한다. 이러한 응용의 이면에는 기계학습의 복잡한 알고리즘이 존재하며, 이는 데이터의 특성과 모델의 성능을 적절히 조정함으로써 가능한 것이다.\n\n결론적으로, 인공지능과 기계학습은 기술적 진보를 이루는 핵심적인 원동력임을 알 수 

In [25]:
user_prompt = f"""
    당신은 대한민국 수학능력시험 국어영역 독서과목 문제를 출제하는 한국교육과정평가원 출제위원입니다.
    다음은 문제와 선택지를 작성할 때 반드시 고려해야 할 기준입니다:
    {results}

    아래 지문을 바탕으로 학생의 이해력, 논리적 추론 능력, 비판적 사고력을 평가할 수 있는 5지선다형 객관식 문항을 1개 작성하세요.

    [지문]
    {llm_result}

    [핵심 논점(출제 의도)]
   지문 생성 시 기억해 두었던 논점을 제시하세요.

    [출제 방향 설정]
    - 반드시 위 '핵심 논점'에 근거하여 문제를 출제하십시오.
    - "{topic}" 관련 출제 경향을 반영할 것.
    - 출제 오류가 발생하지 않도록, 선택지는 모두 지문 내용이나 논리와 명백히 연결되어야 합니다.

		[금지 사항]
		- 지문의 내용과 관련 없는 문항 생성 금지.
		- 비논리적이거나 두 개 이상의 답이 나올 수 있는 문항 생성 금지.
		- 문제와 정답이 명확하지 않은 경우 생성 금지.
		- 하나의 선택지에서는 하나의 정보만 물어볼 것.
		- 지문에 쓰인 명사는 풀어쓰거나 비슷한 표현으로 바꾸지 않고 그대로 쓸 것.

    [문제 생성 출력 형식]
    질문: (학생이 반드시 지문을 읽고 이해해야 풀 수 있는 질문)
    ///
    1. (선택지)
    ///
    2. (선택지)
    ///
    3. (선택지)
    ///
    4. (선택지)
    ///
    5. (선택지)
    ///
    정답: X번 (정답 번호)
    ///
    해설: (정답의 근거와 오답이 틀린 이유를 포함한 상세 해설)
    
    
    """

question_data = generate_text(user_prompt)

In [26]:
question_data

'질문: 지도 학습과 비지도 학습의 차이점으로 올바르게 설명된 것은 무엇인가?\n\n///\n1. 지도 학습은 정답이 없는 데이터 세트를 사용하여 패턴을 인식한다.\n///\n2. 비지도 학습은 정답이 포함된 데이터 세트를 사용하여 모델의 정확성을 향상시킨다.\n///\n3. 지도 학습은 정답을 포함한 데이터 세트를 사용하여 문제를 해결한다.\n///\n4. 비지도 학습에서는 데이터의 군집화와 분류가 불가능하다.\n///\n5. 지도 학습에서는 정답이 없는 데이터 세트를 사용하여 잠재 구조를 찾는다.\n///\n\n정답: 3번\n\n///\n해설: 3번 선택지는 지도 학습의 특성을 정확히 설명하고 있다. 지도 학습에서는 정답(레이블)이 포함된 데이터 세트를 사용하여 학습하고, 이를 통해 모델의 정확성을 향상시킨다. 1번과 5번 선택지는 비지도 학습에 대한 설명으로, 지도 학습이 아닌 비지도 학습에서는 정답이 없는 데이터 세트를 활용하여 패턴을 인식하고 잠재 구조를 찾는다. 2번 선택지는 지도 학습과 비지도 학습의 개념을 혼동하고 있어 틀렸다. 4번 선택지 또한 비지도 학습의 특징을 잘못 이해한 것이다. 비지도 학습은 데이터를 군집화하거나 분류할 수 있는 잠재 구조를 찾는 데 중점을 둔다.'